In [1]:
import os
import re
import glob
import time
import numpy as np
import pandas as pd
from decimal import Decimal

In [2]:
# the overall replacement
def fill_in_na(a):
    if a.strip().startswith("|"):
        a = "NA" + a
    if a.strip().endswith("|"):
        a = a + "NA"
    for i in range(10):
        a = a.replace("||", "|NA|")
    return a

In [3]:
def format_float(a): # process_1
#     a = 1.2345434534
#     b = 2323352525523525.222222222222222
#     c= '%.2E' % Decimal(a)
#     d= '%.2E' % Decimal(b)
#     print(Decimal(c).normalize(), Decimal(d).normalize())
    if a == "NA":
        return "NA"
    else:
        a = float(a)
        b = '%.2E' % Decimal(a)
        return Decimal(b).normalize()

In [4]:
def capitalize_first_letter(a): # process_2
#     print(a[0].upper() + a[1:])
    return a[0].upper() + a[1:]

In [5]:
def correct_spelling(a): # process_3
    # RBP # POSTAR|
    if a.startswith("POSTAR"):
        a = a.replace(a, "POSTAR2|")
    elif a.startswith("starBase"):
        a = a.replace(a, "starBase2|")
    # m6A
    if a.startswith("MeMeRIP-seq"):
        a = a.replace(a, "MeRIP-seq|")
    # splicing sites # Don't function.!!!!!!!
    if a.endswith("bp|"):
        a = re.sub(r"([0-9])b(p)", r"\1 b\2", a.strip("|")) # replace "bp" with " bp"
    # sQTL
    if a.startswith("ExonCorrelation;"):
        a = a.strip("ExonCorrelation;")
    return a

In [22]:
# merge the RBP records with the same GSExxx number;
def format_sample(name_list):
    name_list = list(set(name_list))
    study_list = [x for x in name_list if x.startswith(("GSE", "DRA", "E-MTAB", "SRX", "ENCODE"))]
    sample_list = [x for x in name_list if x.startswith(("GSM", "DRS", "ERR"))]
    formatted_output = (",".join(sorted(study_list)) + ":" + ",".join(sorted(sample_list))).strip(":")
#     formatted_output = "".join(name_list)
#     if len(study_list) > 1:
#         print(study_list)
#     elif len(study_list) < 1:
#         formatted_output = ",".join(sorted(sample_list))
#     else:
#         formatted_output = study_list[0] + ": " + ",".join(sorted(sample_list))
    return formatted_output

in_file = "/home/galaxy/project/m6AQTL/data/data_for_website/final_data/utf8_data/PseudoU.txt"
df = pd.read_table(in_file, sep="\t").fillna("NA")
line_list = df["RBP_Position|RBP_RBP|RBP_Species|RBP_Database|RBP_Study"]
result_list = []
for line in line_list:
    if line != "NA":
        study_dict = {}
        term_list = line.split("@")
        for x in term_list:
            i_key, values = "|".join(x.split("|")[:4]), x.split("|")[-1].replace(":", ",").split(",")
            study_dict[i_key] = study_dict.get(i_key, []) + [x.strip() for x in values]
        new_list = []
        for i_key, values in study_dict.items():
            new_list.append("%s|%s" % (i_key, format_sample(values)))
        new_line = "@".join(new_list)
        result_list.append(new_line)
    else:
        result_list.append("NA")
df["RBP_Position|RBP_RBP|RBP_Species|RBP_Database|RBP_Study"] = result_list

In [23]:
# remove the duplicates of The m6a column
line_list = df["Gene_Name|Ensembl_Gene_ID|Gene_Type|Strand|Supported_sample(s)|PubmedID|source|Tissue"]
result_list = []
for line in line_list:
    if line != "NA":
        study_dict = {}
        term_list = line.split("@")
        new_line = "@".join(list(set(term_list)))
        result_list.append(new_line)
    else:
        result_list.append("NA")
df["Gene_Name|Ensembl_Gene_ID|Gene_Type|Strand|Supported_sample(s)|PubmedID|source|Tissue"] = result_list

In [24]:
df.to_csv("/home/galaxy/project/m6AQTL/data/data_for_website/final_data/utf8_data/PseudoU_raw.txt", sep="\t", index=False)

In [25]:
# Function: As the last process to format or correct the data used for web construction.
# Author: galayx
# Time: 20th of june, 2019
print(time.time())
in_file = "/home/galaxy/project/m6AQTL/data/data_for_website/final_data/utf8_data/PseudoU_raw.txt"
df = pd.read_table(in_file, sep="\t").fillna("NA")
# print(df.head())
conf_dict = {"eQTL":[[4],[5,6],[]], "meQTL":[[6],[7,8],[]], "pQTL":[[5],[6,7],[]], "sQTL":[[7],[8,9],[3]], "circQTL":[[5],[6,7],[]], 
             "GWAS":[[4],[],[]], "COSMIC":[[],[3],[]], "m6A":[[],[7],[6]], "RBP":[[],[2],[3]], 
             "Conservation score":[[0],[],[]], "Spectrum width score":[[0],[],[]], "splicing":[[],[],[5]]} #value(three lists represents three processes)
title_list = [x for x in df.columns if x.split("_")[0] in conf_dict]
result_list = []
for title in title_list:
    QTL_TYPE, conf_list = title.strip().split("_")[0], []
    try:
        conf_list = conf_dict[QTL_TYPE]
    except KeyError:
        print("%s doesn't exit in conf_dict!" % QTL_TYPE)
    print(QTL_TYPE)
#   
    raw_list, modified_list = df[title].tolist(), []
    for line in raw_list:
        if line == "NA":
            modified_list.append("NA")
        else:
            line = str(line).replace("Cis", "cis").replace("Trans", "trans").replace("cells", "cell")
            line = fill_in_na(line)
            term_list = line.split("@")
            for term in term_list: # Three different processing treatment;
                for i_index in conf_list[0]:
                    raw_value = term.split("|")[i_index]
                    line = line.replace(raw_value, str(format_float(raw_value))) # method 1;
                for i_index in conf_list[1]:
                    raw_value = term.split("|")[i_index]
                    line = line.replace(raw_value, capitalize_first_letter(raw_value)) # method 2;
                for i_index in conf_list[2]:
                    raw_value = term.split("|")[i_index].strip() + "|"   # with the saparator at the same time.
                    line = line.replace(raw_value, correct_spelling(raw_value)) # method 3; ???         
            modified_list.append(line)
    print(time.time())
    result_list.append(modified_list)
#
for i in range(len(title_list)):
    df[title_list[i]] = result_list[i]
df.to_csv("/home/galaxy/project/m6AQTL/data/data_for_website/final_data/utf8_data/PseudoU_final.txt", sep="\t", index=False)

1562922897.7177436
Conservation score
1562922897.7420251
Spectrum width score
1562922897.7437978
RBP
1562922897.7517278
splicing
1562922897.7528427
GWAS
1562922897.7536786
eQTL
1562922897.7591097
meQTL
1562922897.759698
pQTL
1562922897.7601488
sQTL
1562922897.7607872
circQTL
1562922897.761371
COSMIC
1562922897.761638


#### TBD： select the first 20th records in each row

In [25]:
# in_file = "/home/galaxy/project/m6AQTL/data/data_for_website_raw.txt"
# df = pd.read_table(in_file, sep="\t")
# df.head()
def stat_records_number(term):
    return len(str(term).split("@"))

col_list = df.columns[2:]
num_array = []
for i_index, values in df.iterrows():
    count_list = []
    for i_col in col_list:
        count_list.append(stat_records_number(values[i_col]))
    num_array.append(count_list)
df_num = pd.DataFrame(num_array, columns=col_list)
print(df_num.head())

   m6A_Gene_Name|m6A_Ensembl_Gene_ID|m6A_Gene_Type|m6A_Strand|m6A_Supported_sample(s)|m6A_PubmedID|m6A_m6A_source|m6A_Tissue  \
0                                                  5                                                                           
1                                                 12                                                                           
2                                                  2                                                                           
3                                                 12                                                                           
4                                                 14                                                                           

   RBP_Position|RBP_RBP|RBP_Species|RBP_Database|RBP_Study  \
0                                                  1         
1                                                  5         
2                                            

In [29]:
# for i_col in col_list:
#     print(i_col.split("_")[0], max(df_num[i_col].tolist()))

# num_list = df_num["clinvar_rs_ID|clinvar_Significant|clinvar_Disease(s)|clinvar_Accession|clinvar_Study_name|clinvar_Review_status"]
# print([i for i in range(len(num_list)) if num_list[i] == 3262])

df["clinvar_rs_ID|clinvar_Significant|clinvar_Disease(s)|clinvar_Accession|clinvar_Study_name|clinvar_Review_status"][7724]

'rs1010541689|Likely benign|not specified|SCV000716479.1|ClinVar:OMIM:HGNC:MedGen:ClinGen|criteria provided, single submitter@rs397517735|Pathogenic|multiple conditions|SCV000286884.5|OMIM:HGNC:Orphanet:MedGen:ClinGen|criteria provided, single submitter@rs397517735|Pathogenic|not provided|SCV000708831.2|OMIM:HGNC:MedGen:HGMD:ClinGen|criteria provided, single submitter@rs397517735|Likely pathogenic|Primary dilated cardiomyopathy|SCV000064418.5|SNOMED CT:EFO:OMIM:HGNC:Orphanet:MedGen:ClinGen|criteria provided, single submitter@rs55972010|Likely benign|Dilated Cardiomyopathy, Dominant|SCV000421131.2|HGNC:MedGen:OMIM:ClinGen|criteria provided, single submitter@rs55972010|Likely benign|Distal myopathy Markesbery-Griggs type|SCV000421132.2|OMIM:HGNC:Orphanet:MedGen:ClinGen|criteria provided, single submitter@rs55972010|Benign|Cardiovascular phenotype|SCV000317848.4|HGNC:MedGen:OMIM:ClinGen|criteria provided, single submitter@rs55972010|Benign|not specified|SCV000315580.1|HGNC:MedGen:OMIM:Cli

In [27]:
# a = ["A", "B", "C"]
# ", ".join(a)
a = "a:GSE83923:"
a.strip(":")

'a:GSE83923'